In [1]:
# from utils.compression import compress_directories, decompress_directories

# for base_path, batch_type in data_partition.items():
#     decompress_directories(
#         source_path=f'downloaded_data/{base_path}',
#         destination_path=f'data/{base_path}',
#         checksum_file_name="checksums.txt",
#         delete_compressed_files=True,
#         num_workers=26
#     )

In [2]:
# del session.logger
# del session

from pickle import NONE
from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
import multiprocessing

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "schoffelen2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=False,
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 40)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="standard",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/ablation/downsample200/40",
    clear_cache=False,
    cache_name="cache",
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
SimpleConv initialized with 8448160 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 7973376


2024-12-22 12:04:02,008	INFO worker.py:1821 -- Started a local Ray instance.
Training Epoch 1:   0%|          | 0/135 [00:00<?, ?it/s]

x shape: torch.Size([256, 384, 800]) after initial batch norm, linear and conditional layers
module_idx: 0, input shape: torch.Size([256, 384, 800])
after module: 0, output shape: torch.Size([256, 384, 400])
after glu: 0, output shape: torch.Size([256, 384, 400])
module_idx: 1, input shape: torch.Size([256, 384, 400])
after module: 1, output shape: torch.Size([256, 384, 200])
after glu: 1, output shape: torch.Size([256, 384, 200])
module_idx: 2, input shape: torch.Size([256, 384, 200])
after module: 2, output shape: torch.Size([256, 384, 100])
after glu: 2, output shape: torch.Size([256, 384, 100])
module_idx: 3, input shape: torch.Size([256, 384, 100])
after module: 3, output shape: torch.Size([256, 384, 50])
after glu: 3, output shape: torch.Size([256, 384, 50])
module_idx: 4, input shape: torch.Size([256, 384, 50])
after module: 4, output shape: torch.Size([256, 384, 25])
after glu: 4, output shape: torch.Size([256, 384, 25])
module_idx: 5, input shape: torch.Size([256, 384, 25])
af

RuntimeError: The size of tensor a (400) must match the size of tensor b (13) at non-singleton dimension 2

## 200Hz, 40